In [284]:
import sys
!pip install pandas
!pip install numpy
!pip install datapane
!pip install plotly
!pip install pymysql

In [285]:
# import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp
import pymysql
import warnings
import pymysql
import sys
import os

warnings.filterwarnings('ignore')

In [286]:
host = 'mis686group5.cd3st0ffdll2.us-east-2.rds.amazonaws.com'
user = 'admin'
password = '12345678'
database = 'Cafe'

connection = pymysql.connect(host=host, user=user, passwd=password, database=database)
with connection:
    cur = connection.cursor()
    cur.execute("SELECT VERSION()")
    version = cur.fetchone()
    print("Database version: {} ".format(version[0]))

Database version: 8.0.33 


In [287]:
def run_query(q: str) -> pd.DataFrame:
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        return pd.read_sql(q, conn)


def run_command(c: str):
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        conn.isolation_level = None
        conn.execute(c)

In [288]:
def show_tables() -> pd.DataFrame:
    q = """
          SELECT table_name
          FROM information_schema.tables
          WHERE table_schema = 'Cafe' AND table_type = 'BASE TABLE';
        """
    return run_query(q)


def get_table_row_count(tablename) -> int:
    q = (
        """
        SELECT
            COUNT(1)
        FROM %s;
        """
        % tablename
    )
    return run_query(q)["COUNT(1)"][0]


tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables['TABLE_NAME']]

tables

,TABLE_NAME,row_count
0,Customer,30
1,Customer_Review,30
2,Ingredient,30
3,Inventory,30
4,Menu_Item,30
5,Order_Item,30
6,Order_Promotion,30
7,Orders,30
8,Promotion,6
9,Staff,30


In [289]:
#category distribution
category_dist = """
SELECT
    Type,
    COUNT(Menu_Item_ID) AS "Number of Items"
FROM
    Menu_Item
GROUP BY
    Type;
"""
run_query(category_dist)

,Type,Number of Items
0,Coffee,8
1,Tea,4
2,Frappuccino,1
3,Bakery,3
4,Breakfast,3
5,Salad,1
6,Sandwich,1
7,Wrap,1
8,Smoothie,1
9,Quiche,1


In [313]:
#visualize the distribution
category_distribution = run_query(category_dist)

# Create a pie chart
category_distribution_plot = px.pie(
    category_distribution,
    values='Number of Items',
    names='Type',
    title='Distribution of Menu Items by Category'
)

# category_distribution_plot.show()
category_distribution_report = dp.Report(dp.Plot(category_distribution_plot))
category_distribution_report


[06:13:32] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://5b512bb55c"/>
</View>



In [291]:
# Example Analysis 2: Inventory Status
inventory_status_query = """
SELECT
    i.Name ingredient_name,
    iv.Quantity inventory_quantity
FROM Ingredient i
JOIN Inventory iv ON i.Ingredient_ID = iv.Ingredient_ID
ORDER BY inventory_quantity DESC
"""

run_query(inventory_status_query)




,ingredient_name,inventory_quantity
0,Milk,300
1,Brownie Mix,300
2,Romaine Lettuce,300
3,Eggnog,250
4,Iced Coffee Concentrate,250
5,Steamed Milk,200
6,Cream,200
7,Mixed Fruits,200
8,Tomato,200
9,Blueberries,200


In [292]:
# Example Analysis 3: Monthly Revenue
monthly_revenue_query = """
SELECT
    DATE_FORMAT(Order_Date, '%Y-%m') month,
    SUM(Total_Amount) total_revenue
FROM Orders
GROUP BY 1
ORDER BY 1
"""
run_query(monthly_revenue_query)


,month,total_revenue
0,2023-01,641.25
1,2023-02,375.25
2,2023-03,301.25
3,2023-04,340.20
4,2023-05,201.25
5,2023-06,180.25
6,2023-07,220.95
7,2023-08,230.50
8,2023-09,190.25
9,2023-10,291.25


In [293]:
import plotly.express as px
import datapane as dp

# Query for Top 10 Customers Loyalty Points
top_10_loyalty_query = """
SELECT
    Customer_ID,
    Name customer_name,
    Loyalty_Points
FROM Customer
ORDER BY Loyalty_Points DESC
LIMIT 10
"""
# Running the query
top_10_loyalty_df = run_query(top_10_loyalty_query)

# Bar chart for Top 10 Customers Loyalty Points
fig_top_loyalty = px.bar(
    top_10_loyalty_df,
    x='Loyalty_Points',
    y='customer_name',
    orientation='h',
    text='Loyalty_Points',  # Add this line to display values on bars
    title='Top 10 Customers by Loyalty Points'
)
fig_top_loyalty.update_layout(xaxis_title='Loyalty Points', yaxis_title='Customer Name')

# Showing the chart using Datapane
dp.Report(dp.Table(top_10_loyalty_df, caption="Top 10 Customers by Loyalty Points"), dp.Plot(fig_top_loyalty))


[05:58:47] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://6f58a22205" caption="Top 10 Customers by Loyalty Points"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://c7d902446f"/>
</View>



In [294]:
import plotly.express as px
import datapane as dp

# Query for Staff Skills
staff_skills_query = """
SELECT
    Name,
    Coffee_Skills_Level,
    Register_Management_Skills,
    Management_Experience,
    Leadership_Skills
FROM Staff;
"""

# Running the query
staff_skills_data = run_query(staff_skills_query)

# Creating Radar Chart for Staff Skills
fig_staff_skills = px.line_polar(
    staff_skills_data,
    r=staff_skills_data.iloc[:, 1:].mean(),
    theta=staff_skills_data.columns[1:],
    line_close=True,
    title='Average Staff Skills'
)



In [295]:
# Now let's create a bar chart for total order amounts by staff
order_amounts_by_staff_query = """
SELECT
    s.Name,
    SUM(o.Total_Amount) as Total_Order_Amount
FROM `Orders` o
JOIN Staff s ON o.Staff_ID = s.Staff_ID
GROUP BY s.Name;

"""

# Running the query
order_amounts_by_staff_data = run_query(order_amounts_by_staff_query)

# Bar plot for total order amounts by staff
fig_order_amounts_by_staff = px.bar(
    order_amounts_by_staff_data,
    x='Name',
    y='Total_Order_Amount',
    title='Total Order Amounts by Staff',
    labels={'Total_Order_Amount': 'Total Order Amount'}
)
# Showing the visualizations using Datapane
dp.Report(
    # dp.Table(staff_skills_data, caption="Staff Skills Data"),
    dp.Plot(fig_staff_skills),
    # dp.Table(order_amounts_by_staff_data, caption="Order Amounts by Staff Data"),
    dp.Plot(fig_order_amounts_by_staff)
)


[05:58:48] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://73eff7f7d4"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://9c587a13c5"/>
</View>



In [296]:
import plotly.express as px

# Query for Monthly Revenue by Staff
staff_monthly_revenue_query = """
SELECT
    s.Name as staff_name,
    DATE_FORMAT(o.Order_Date, '%Y-%m') as month,
    SUM(o.Total_Amount) as total_revenue
FROM `Orders` o
JOIN Staff s ON o.Staff_ID = s.Staff_ID
GROUP BY 1, 2
ORDER BY 1, 2;
"""

# Running the query
staff_monthly_revenue_df = run_query(staff_monthly_revenue_query)

# Line chart for Monthly Revenue by Staff
fig_staff_monthly_revenue = px.line(
    staff_monthly_revenue_df,
    x='month',
    y='total_revenue',
    color='staff_name',
    title='Monthly Revenue by Staff',
    markers=True
)
fig_staff_monthly_revenue.update_layout(xaxis_title='Month', yaxis_title='Total Revenue (in $)')

# Showing the chart using Datapane
dp.Report(dp.Table(staff_monthly_revenue_df, caption="Monthly Revenue by Staff"), dp.Plot(fig_staff_monthly_revenue))


[05:58:48] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://4473225297" caption="Monthly Revenue by Staff"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://8cf109e5e6"/>
</View>



In [297]:
import plotly.express as px

# Query for Monthly Revenue by Staff
staff_monthly_revenue_query = """
SELECT
    s.Name as staff_name,
    SUM(o.Total_Amount) as total_revenue
FROM `Orders` o
JOIN Staff s ON o.Staff_ID = s.Staff_ID
GROUP BY 1
ORDER BY 2 DESC;
"""

# Running the query
staff_total_revenue_df = run_query(staff_monthly_revenue_query)

# Select the top 10 staff based on total revenue
top_10_staff = staff_total_revenue_df.head(10)

# Bar chart for Total Revenue by Staff
# fig_total_revenue_by_staff = px.bar(
#     top_10_staff,
#     x='staff_name',
#     y='total_revenue',
#     title='Total Revenue by Staff',
#     labels={'total_revenue': 'Total Revenue (in $)'}
# )

# Showing the chart using Datapane along with a table
dp.Report(dp.Table(staff_total_revenue_df, caption="Total Revenue by Staff"))


[05:58:49] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://5ecabd2166" caption="Total Revenue by Staff"/>
</View>



In [316]:
import plotly.express as px
import datapane as dp

# Query for Staff Counts by Role Type
staff_role_counts_query = """
SELECT
    Role_Type,
    COUNT(*) as Staff_Count
FROM Staff
GROUP BY 1
"""

# Running the query
staff_role_counts_df = run_query(staff_role_counts_query)

# Donut chart for Staff Counts by Role Type
fig_staff_role_counts_donut = px.pie(
    staff_role_counts_df,
    names='Role_Type',
    values='Staff_Count',
    hole=0.4,  # Set the size of the hole in the middle
    title='Staff Count by Role Type (Donut Chart)'
)

# Showing the chart using Datapane
dp.Report(dp.Table(staff_role_counts_df, caption="Staff Count by Role Type"), dp.Plot(fig_staff_role_counts_donut))


[06:37:50] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://300fb310c7" caption="Staff Count by Role Type"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://989aa5ca22"/>
</View>



In [299]:
import plotly.express as px
import datapane as dp

# Query for Customer Loyalty Points Distribution
loyalty_points_distribution_query = """
SELECT
    Loyalty_Points
FROM
    Customer;
"""

# Running the query
loyalty_points_distribution_df = run_query(loyalty_points_distribution_query)

# Histogram for Customer Loyalty Points Distribution
fig_loyalty_points_distribution = px.histogram(
    loyalty_points_distribution_df,
    x='Loyalty_Points',
    nbins=20,
    title='Customer Loyalty Points Distribution',
    labels={'Loyalty_Points': 'Loyalty Points', 'count': 'Frequency'},
    opacity=0.7,  # Set opacity for better visualization
    color_discrete_sequence=['indianred'],  # Change the color
    histnorm='percent',  # Show histogram as a percentage
)

# Adjusting layout parameters
fig_loyalty_points_distribution.update_layout(
    autosize=False,
    width=800,  # Set the width of the plot
    height=500,  # Set the height of the plot
    margin=dict(l=50, r=50, b=50, t=50, pad=4),  # Adjust margins for better spacing
)

# Showing the chart using Datapane
dp.Report(dp.Table(loyalty_points_distribution_df, caption="Customer Loyalty Points Distribution"), dp.Plot(fig_loyalty_points_distribution))


[05:58:49] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://bdbe9bc5e9" caption="Customer Loyalty Points Distribution"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://95d18559ce"/>
</View>



In [300]:
# Query for Inventory Status
inventory_status_query = """
SELECT
    i.Name ingredient_name,
    iv.Quantity inventory_quantity
FROM Ingredient i
JOIN Inventory iv ON i.Ingredient_ID = iv.Ingredient_ID
ORDER BY inventory_quantity DESC
"""

# Running the query
inventory_status_df = run_query(inventory_status_query)

# Pie chart for Inventory Status
fig_inventory_status_pie = px.pie(inventory_status_df, names='ingredient_name', values='inventory_quantity', title='Inventory Status')
fig_inventory_status_pie.update_layout(width=1000, height=900)

# Showing the chart using Datapane
# dp.Report(dp.Table(inventory_status_df, caption="Inventory Status"), dp.Plot(fig_inventory_status))
dp.Report(dp.Plot(fig_inventory_status_pie))


[05:58:50] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://7ecf68c5df"/>
</View>



In [301]:
import plotly.express as px
import datapane as dp

# Define the threshold for low inventory
low_inventory_threshold = 70  # Adjust this threshold as needed

# Query for Inventory Status
inventory_status_query = """
SELECT
    i.Name ingredient_name,
    iv.Quantity inventory_quantity
FROM Ingredient i
JOIN Inventory iv ON i.Ingredient_ID = iv.Ingredient_ID
ORDER BY inventory_quantity DESC
"""

# Running the query
inventory_status_df = run_query(inventory_status_query)

# Create a new column for color-coding
inventory_status_df['status'] = ['Low Inventory' if qty < low_inventory_threshold else 'Adequate Inventory' for qty in inventory_status_df['inventory_quantity']]

# Bar chart for Inventory Status
fig_inventory_status = px.bar(
    inventory_status_df,
    x='ingredient_name',
    y='inventory_quantity',
    color='status',
    text='inventory_quantity',  # Add this line to display values on bars
    title='Inventory Status',
    labels={'inventory_quantity': 'Inventory Quantity'},
)

# Customize legend labels
fig_inventory_status.for_each_trace(lambda t: t.update(name=t.name.replace("status=", "")))

# Showing the chart using Datapane
dp.Report(dp.Plot(fig_inventory_status))


[05:58:50] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://5cf7ddbc99"/>
</View>



In [302]:
import pandas as pd
import plotly.express as px
import datapane as dp

# Assuming you have a DataFrame named 'customer_reviews' in your database
customer_reviews_query = """
SELECT
    Review_Date,
    Rating
FROM
    Customer_Review;
"""

# Running the query
customer_reviews_df = run_query(customer_reviews_query)

# Converting 'review_date' to datetime
customer_reviews_df['Review_Date'] = pd.to_datetime(customer_reviews_df['Review_Date'])

# Grouping by 'review_date' and calculating the mean rating
daily_avg_ratings = customer_reviews_df.groupby('Review_Date').mean().reset_index()

# Plotting Average Customer Ratings Over Time using Plotly Express with markers
fig_avg_ratings = px.line(
    daily_avg_ratings,
    x='Review_Date',
    y='Rating',
    title='Average Customer Ratings Over Time',
    labels={'Rating': 'Average Rating'},
    markers=True,  # Include markers on the line
)

# Showing the chart using Datapane
dp.Report(dp.Table(daily_avg_ratings, caption="Average Customer Ratings Over Time"), dp.Plot(fig_avg_ratings))


[05:58:50] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://a25bff7421" caption="Average Customer Ratings Over Time"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://703c3521f9"/>
</View>



In [303]:
import pandas as pd
import plotly.express as px
import datapane as dp

# Assuming you have a DataFrame named 'orders' and 'order_promotions' in your database
promotion_impact_query = """
SELECT
    o.Order_ID,
    o.Total_Amount,
    op.Promotion_ID,
    p.Discount_Amount,
    p.Name
FROM
    Orders o
LEFT JOIN
    Order_Promotion op ON o.Order_ID = op.Order_ID
LEFT JOIN
    Promotion p ON op.Promotion_ID = p.Promotion_ID;
"""

# Running the query
promotion_impact_df = run_query(promotion_impact_query)

# Creating a new column for promotion impact
promotion_impact_df['promo_effect'] = promotion_impact_df['Total_Amount'] * (1 - promotion_impact_df['Discount_Amount'] / 100)

# Grouping by promotion_id and summing promo_effect
promo_effect = promotion_impact_df.groupby('Name')['promo_effect'].sum().reset_index()

# Bar plot for Promotion Impact on Sales using Plotly Express
fig = px.bar(promo_effect, x='Name', y='promo_effect', title='Promotion Impact on Sales',
             labels={'promo_effect': 'Total Sales with Promotion'},
             text='promo_effect',  # Add this line to display values on bars
)

# Display the plot
# fig.show()

# Create a Datapane report
promotions_effect_report = dp.Report(dp.Plot(fig, caption='Promotion Impact on Sales'))
promotions_effect_report


[05:58:51] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://2f0b95d20e" caption="Promotion Impact on Sales"/>
</View>



In [304]:
# Query for Monthly Revenue
monthly_revenue_query = """
SELECT
    DATE_FORMAT(Order_Date, '%Y-%m') month,
    SUM(Total_Amount) total_revenue
FROM Orders
GROUP BY 1
ORDER BY 1
"""

# Running the query
monthly_revenue_df = run_query(monthly_revenue_query)

# Line chart for Monthly Revenue
fig_monthly_revenue = px.line(monthly_revenue_df, x='month', y='total_revenue', title='Monthly Revenue', markers=True)
fig_monthly_revenue.update_layout(xaxis_title='Month', yaxis_title='Total Revenue (in $)')

# Showing the chart using Datapane
dp.Report(dp.Table(monthly_revenue_df, caption="Monthly Revenue"), dp.Plot(fig_monthly_revenue))


[05:58:51] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://8d086548b5" caption="Monthly Revenue"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://f61f77e4b3"/>
</View>



In [305]:
import pandas as pd
import plotly.express as px
import datapane as dp

# Assuming you have a DataFrame named 'orders' and 'order_promotions' in your database
sales_query = """
SELECT
    o.Order_Date,
    o.Total_Amount,
    op.Promotion_ID
FROM
    Orders o
LEFT JOIN
    Order_Promotion op ON o.Order_ID = op.Order_ID;
"""

# Running the query
sales_df = run_query(sales_query)

# Converting 'order_date' to datetime
sales_df['Order_Date'] = pd.to_datetime(sales_df['Order_Date'])

# Extracting month and year
sales_df['month_year'] = sales_df['Order_Date'].dt.to_period("M").astype(str)  # Convert Period to string

# Creating a new column to indicate whether the order has a promotion or not
sales_df['has_promotion'] = sales_df['Promotion_ID'].notnull()

# Grouping by month and calculating total sales and sales with promotions
monthly_sales = sales_df.groupby(['month_year', 'has_promotion']).sum()['Total_Amount'].unstack().fillna(0).reset_index()

# Rename columns for better naming in legend
monthly_sales.columns = ['Month', 'No Promotion', 'Promotion']

# Melt the DataFrame to create a 'Promotion Type' column for color
monthly_sales_melted = pd.melt(monthly_sales, id_vars=['Month'], var_name='Promotion Type', value_name='Total Sales')

# Add values to the stacked bar chart using Plotly Express
fig = px.bar(
    monthly_sales_melted,
    x='Month',
    y='Total Sales',
    color='Promotion Type',
    color_discrete_map={'No Promotion': 'blue', 'Promotion': 'orange'},
    labels={'Total Sales': 'Total Sales'},
    title='Monthly Sales Comparison with Promotions',
    barmode='stack',  # Set the barmode to 'stack' for stacked bars
    text='Total Sales',  # Add values to the bars
)

fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales', xaxis_tickangle=-45)

# Create Datapane report
promotion_report = dp.Report(dp.Plot(fig))
promotion_report


[05:58:51] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://4e61534c82"/>
</View>



In [306]:
import pandas as pd
import plotly.express as px
import datapane as dp

# Query for Total Quantity Sold by Menu Item and Month
menu_item_quantity_query = """
WITH menu_item_sales AS (
    SELECT
        oi.Order_Item_ID,
        mi.Name AS menu_item_name,
        DATE_FORMAT(o.Order_Date, '%Y-%m') AS month,
        SUM(oi.Quantity) AS total_quantity_sold
    FROM Order_Item oi
    JOIN Menu_Item mi ON oi.Order_Item_ID = mi.Menu_Item_ID
    JOIN Orders o ON oi.Order_id = o.Order_id
    GROUP BY 1, 2, 3
)

SELECT
    mis.menu_item_name,
    mis.month,
    SUM(mis.total_quantity_sold) AS total_quantity_sold
FROM menu_item_sales mis
GROUP BY 1, 2
ORDER BY 1, 2;
"""

# Running the query
menu_item_quantity_df = run_query(menu_item_quantity_query)
menu_item_quantity_df["month"] = pd.to_datetime(menu_item_quantity_df["month"])


# Showing the chart using Datapane
dp.Report(dp.Table(menu_item_quantity_df, caption="Total Quantity Sold by Menu Item and Month"))


[05:58:52] [DEBUG] <View version="1" fragment="false">
  <Table type="application/vnd.datapane.table+html" src="ref://1b64303ad6" caption="Total Quantity Sold by Menu Item and Month"/>
</View>



In [307]:
import plotly.express as px

# Assuming menu_item_quantity_df has the columns 'menu_item_name', 'month', and 'total_quantity_sold'
menu_item_quantity_df["month"] = pd.to_datetime(menu_item_quantity_df["month"]).dt.strftime('%Y-%m')  # Format to 'YYYY-MM'

# Bar chart for Total Quantity Sold by Menu Item
fig_max_items_sold = px.bar(
    menu_item_quantity_df,
    x='menu_item_name',
    y='total_quantity_sold',
    color='month',
    title='Total Quantity Sold by Menu Item',
    labels={'total_quantity_sold': 'Total Quantity Sold'},
    category_orders={"month": sorted(menu_item_quantity_df["month"].unique())}  # Sort the months
)

fig_max_items_sold.update_layout(xaxis_title='Menu Item', yaxis_title='Total Quantity Sold')

# Show the chart
fig_max_items_sold.show()


In [308]:
import pandas as pd
import plotly.express as px
import datapane as dp

# Assuming menu_item_quantity_df has the columns 'menu_item_name', 'month', and 'total_quantity_sold'
menu_item_quantity_df["month"] = pd.to_datetime(menu_item_quantity_df["month"]).dt.strftime('%Y-%m')  # Format to 'YYYY-MM'

# Bar chart for Total Quantity Sold by Menu Item
fig_max_items_sold = px.bar(
    menu_item_quantity_df,
    x='month',
    y='total_quantity_sold',
    color='menu_item_name',  # Use 'menu_item_name' as the legend
    title='Total Quantity Sold by Menu Item',
    labels={'total_quantity_sold': 'Total Quantity Sold'},
    category_orders={"month": sorted(menu_item_quantity_df["month"].unique())}  # Sort the months
)

fig_max_items_sold.update_layout(xaxis_title='Month and Year', yaxis_title='Total Quantity Sold')

# Create Datapane report
quanity_sold_report = dp.Report(dp.Plot(fig_max_items_sold))
quanity_sold_report


[05:58:52] [DEBUG] <View version="1" fragment="false">
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://c6df10fec0"/>
</View>



In [309]:
#data pane
customers_count = tables[tables["TABLE_NAME"] == "Customer"]["row_count"].values[0]
employees_count = tables[tables["TABLE_NAME"] == "Staff"]["row_count"].values[0]
total_sales_count = tables[tables["TABLE_NAME"] == "Orders"]["row_count"].values[0]

In [317]:
# Creating a Data Panel view with a title
r = dp.View(
    dp.Text("Cafe Management System Dashboard"),  # Adding a title
    dp.Group(
        dp.BigNumber(heading="Total Customers", value=customers_count),
        dp.BigNumber(heading="Total Employees", value=employees_count),
        dp.BigNumber(heading="Total Sales Count", value=total_sales_count),
        columns=3,
        name="Summary",
    ),
    dp.Plot(fig_top_loyalty),
    dp.Plot(fig_inventory_status),
    dp.Group(dp.Plot(fig_monthly_revenue), category_distribution_report, columns=2),
    dp.Plot(fig_avg_ratings),
    quanity_sold_report,
    dp.Group(promotion_report, promotions_effect_report, columns=2),
    dp.Group(dp.Plot(fig_staff_role_counts_donut), dp.Plot(fig_order_amounts_by_staff), columns=2),
    dp.Plot(fig_staff_monthly_revenue),
    dp.Table(staff_total_revenue_df, caption="Total Revenue by Staff")
)

dp.enable_logging()

# Saving the report
dp.save_report(r, "cafe_report.html", open=True,
    formatting=dp.Formatting(
        light_prose=False,
        accent_color="orange",
        text_alignment=dp.TextAlignment.CENTER,
        font=dp.FontChoice.MONOSPACE,
        width=dp.Width.MEDIUM,
    ))


[06:38:07] [WARNING] Configuring datapane logging in library mode
[06:38:07] [WARNING] Reconfiguring datapane logger when running as LIBRARY
[06:38:07] [DEBUG] <View version="1" fragment="false">
  <Text><![CDATA[Cafe Management System Dashboard]]></Text>
  <Group name="Summary" columns="3" valign="top">
    <BigNumber heading="Total Customers" value="30" is_positive_intent="false" is_upward_change="false"/>
    <BigNumber heading="Total Employees" value="30" is_positive_intent="false" is_upward_change="false"/>
    <BigNumber heading="Total Sales Count" value="30" is_positive_intent="false" is_upward_change="false"/>
  </Group>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://c7d902446f"/>
  <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://5cf7ddbc99"/>
  <Group columns="2" valign="top">
    <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://f61f77e4b3"/>
    <Group columns="1" 

App saved to ./cafe_report.html

In [318]:
#export the report
import IPython
IPython.display.HTML(filename='cafe_report.html')